In [13]:
from langchain.agents import AgentExecutor,create_react_agent,Tool,create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from pydantic import BaseModel,Field
from typing import Dict,List
from langchain_core.output_parsers import PydanticOutputParser
import json
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate


from langchain import hub
from langchain.chains import LLMChain
from concurrent.futures.thread import ThreadPoolExecutor
from concurrent.futures import as_completed
import time
import json
from pydantic import BaseModel,Field
from typing import List,Dict

In [2]:
llm=ChatOpenAI(
    # model="mistralai/mistral-nemo-instruct-2407",
    # model="qwen/qwen3-4b-thinking-2507",
    model="hermes-3-llama-3.2-3b",
    base_url="http://127.0.0.1:1239/v1/",
    api_key="dewfe",
)

In [34]:
class parser(BaseModel):
    content:dict=Field(description="response of llm")
    
oup_parser=PydanticOutputParser(pydantic_object=parser)

print(oup_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"content": {"additionalProperties": true, "description": "response of llm", "title": "Content", "type": "object"}}, "required": ["content"]}
```


In [50]:
import tkinter as tk

# Create the main window
def helper():
    root = tk.Tk()
    root.title("Hello Tkinter")
    
    # Add a label widget
    label = tk.Label(root, text="Hello, World!")
    label.pack()
    
    # Run the application
    root.mainloop()

helper()

In [53]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool

# th_pl_exec=ThreadPoolExecutor(max_workers=3)

@tool
def pollGenerator(context: str) -> dict:
    """This tool generates a poll for the room to increase engagement.
    Return a dictionary 
    """

    template=ChatPromptTemplate.from_messages([
        ("system",
        """
        you are a poll generator 
        based on context:{context} generate poll

        output in format:{format}
        """)   
    ])

    llm_struct=llm.with_structured_output(parser)
    chain=template|llm_struct
    result=chain.invoke({"context":context,"format":oup_parser.get_format_instructions()}).content
    print(f"Result:{result}")
    #trigger tkinter
    helper()
    
    return result
    

def create_room_agent():
    tools = [pollGenerator]

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert assistant that makes chat rooms more interactive and viral."),
        ("human", 
        """Room Name: {room_name}
        Room Description: {room_desc}
        Recent Chats: {chats}

        based on provided tools call the correct one:
        {tool_names}

        give tool output also
        Task: {input}
        Reasoning:{agent_scratchpad}

        
        """)
    ])

    # ✅ Correct agent constructor
    agent = create_tool_calling_agent(llm, tools, prompt,)

    # ✅ Executor to run agent
    executor = AgentExecutor(agent=agent, tools=tools,max_iterations=1)#,verbose=True)
    return executor


def main():
    agent_executor = create_room_agent()
    
    room_data = {
        "room_name": "Python Developers Hub",
        "room_desc": "A community for Python enthusiasts to share code and learn",
        "chats": "User1: Anyone know good Python resources?\nUser2: I'm working on a Django project",
        "input": "Generate a poll for this room",
        "tool_names":["pollGenerator"]
    }

    result = agent_executor.invoke(room_data)
    # print("\n✅ AGENT RESULT:", result)


if __name__ == "__main__":
    time_start=time.time()
    main()
    print(f"TIME TAKEN:{time.time()-time_start}")


Result:{'prompt': 'Generate a poll based on the context: Python Developers Hub generate poll', 'max_input_length': 1024, 'stop': ['yes', 'no', 'maybe', 'definitely not', 'probably', 'possibly', 'likely', 'unlikely'], 'temperature': 0.5, 'top_k': -1, 'n': 10}
TIME TAKEN:58.38736176490784
